In [26]:
# Libraries used for analysis 

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go

In [2]:
# Files upload 

rest = pd.read_csv("FOOD_CATEGORY_restaurants_output.csv")
menus = pd.read_csv("FOOD_CATEGORY_restaurant_menus_output.csv")

In [3]:
# Number of restaurants in menus file
unique_rest = menus["restaurant_id"].nunique()

print(unique_rest)

550


In [4]:
# Unique categories in menus
unique_categories = menus["category"].value_counts()

print(unique_categories)

Sides                    2253
Picked for you           2231
Drinks                   1839
Tacos                    1426
Burritos                 1391
                         ... 
Champaing Sparkling         1
Premium Margarita Bar       1
Chophouse                   1
Community Kitchen.          1
House Made Drinks           1
Name: category, Length: 1570, dtype: int64


In [5]:
# Unique meals in menus

unique_meal = menus["name"].value_counts()

print((unique_meal).head(10))

Guacamole                       203
Quesadilla                      163
Cheese Quesadilla               146
Nachos                          122
Burrito                         121
Cinnamon Twists                 120
Chips and Salsa                  98
Chicken Quesadilla               96
Black Beans                      95
Chips and Nacho Cheese Sauce     95
Name: name, dtype: int64


In [6]:
# Remove the "USD" string and convert the 'price' column to a numeric data type
menus['price'] = menus['price'].str.replace(' USD', '').astype(float)

In [7]:
# Merge two data sets
merged_data = rest.merge(menus, left_on="rm_restaurant_id", right_on="restaurant_id")

In [8]:
meals_in_all_cities = merged_data.groupby('name')['r_city'].nunique()

print(meals_in_all_cities)

name
  Burrito.with carne asada    1
  Taco Cochinita Pibil        1
 Add Sour Cream               1
 Amarillo.*                   1
 Arrachera                    1
                             ..
wario lopez                   1
white Rice                    1
whole Chicken (halal)         1
yuca.                         1
À la Carte                    1
Name: r_city, Length: 15123, dtype: int64


In [13]:
# Filter data only for meals in all cities
meals_in_all_cities = merged_data.groupby('name')['r_city'].nunique()
meals_all_cities = merged_data[merged_data['name'].isin(meals_in_all_cities)]

# Find the 10 most popular meals from the menus which are present in all cities
popular_meals = meals_all_cities['name'].value_counts().nlargest(10).index.tolist()

# Filter data only for the most popular meals
most_popular_meals = merged_data[merged_data['name'].isin(popular_meals)]

# Check the avg prices for those dishes in each city
most_popular_meals_prices = most_popular_meals.groupby(['r_city', 'name'])['price'].mean().reset_index()

# Round the mean price to 2 decimal places
most_popular_meals_prices['price_mean'] = most_popular_meals_prices['price'].round(2)
most_popular_meals_prices.drop(columns='price', inplace=True)

# Results and comments
print(most_popular_meals_prices)


Empty DataFrame
Columns: [r_city, name, price_mean]
Index: []


In [14]:
# Find meals present in all cities
unique_cities = merged_data['r_city'].nunique()
meals_in_all_cities = merged_data.groupby('name')['r_city'].nunique()
meals_in_all_cities = meals_in_all_cities[meals_in_all_cities == unique_cities].index.tolist()

# Filter data only for meals in all cities
meals_all_cities = merged_data[merged_data['name'].isin(meals_in_all_cities)]

# Find the 10 most popular meals from the menus which are present in all cities
popular_meals = meals_all_cities['name'].value_counts().nlargest(10).index.tolist()

# Filter data only for the most popular meals
most_popular_meals = merged_data[merged_data['name'].isin(popular_meals)]

# Check the avg prices for those dishes in each city
most_popular_meals_prices = most_popular_meals.groupby(['r_city', 'name'])['price'].mean().reset_index()

# Round the mean price to 2 decimal places
most_popular_meals_prices['price_mean'] = most_popular_meals_prices['price'].round(2)
most_popular_meals_prices.drop(columns='price', inplace=True)

# Results and comments
print(most_popular_meals_prices)

           r_city                name  price_mean
0       arlington         Black Beans        2.32
1       arlington             Burrito       11.52
2       arlington         Carne Asada       21.91
3       arlington   Cheese Quesadilla        5.23
4       arlington  Chicken Quesadilla        8.94
..            ...                 ...         ...
75  washington dc           Guacamole        8.84
76  washington dc              Nachos       10.38
77  washington dc          Quesadilla       11.47
78  washington dc          Sour Cream        2.08
79  washington dc          Taco Salad       12.24

[80 rows x 3 columns]


In [16]:
# Check number of unique names of the meals in the most_popular_meals_prices data frame
unique_meal_names = most_popular_meals_prices['name'].unique()
num_unique_meals = len(unique_meal_names)
print(f"Number of unique meals in most_popular_meals_prices: {num_unique_meals}")


Number of unique meals in most_popular_meals_prices: 10


In [15]:
# Check if we have 10 meals for every city
all_meals = set(most_popular_meals['name'].unique())
for r_city, city_data in most_popular_meals_prices.groupby("r_city"):
    city_meals = set(city_data['name'].unique())
    missing_meals = all_meals - city_meals
    print(f"In {r_city}, {len(city_meals)} meals are present. Missing meals: {missing_meals}")

In arlington, 10 meals are present. Missing meals: set()
In austin, 10 meals are present. Missing meals: set()
In birmingham, 10 meals are present. Missing meals: set()
In milwaukee, 10 meals are present. Missing meals: set()
In portland, 10 meals are present. Missing meals: set()
In salt lake city, 10 meals are present. Missing meals: set()
In spokane, 10 meals are present. Missing meals: set()
In washington dc, 10 meals are present. Missing meals: set()


In [30]:
import plotly.express as px

# Create an interactive bar plot using Plotly
fig = px.bar(most_popular_meals_prices, x='r_city', y='price_mean', color='name', barmode='group', text='price_mean',
             title='Average Prices of 10 Most Popular Menu Selections', labels={'r_city': 'City', 'price_mean': 'Average Price'})

# Update the layout for better readability
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': most_popular_meals_prices['r_city'].unique()})
fig.update_traces(texttemplate='%{text:.2f}', textposition='inside')
fig.update_yaxes(title='Average Price', tickprefix='$')

# Show the plot
fig.show()

In [31]:
# Save the interactive plot as an HTML file
pio.write_html(fig, file="most_popular_meals_prices.html", auto_open=True)